# Document Question Answering with local persistence

An example of using Chroma DB and LangChain to do question answering over documents, with a locally persisted database. 
You can store embeddings and documents, then use them again later.

In [1]:
from tqdm.notebook import tqdm_notebook as notebook_tqm
from dotenv import load_dotenv

# Load the environment variables from .env
load_dotenv()

True

In [2]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings.huggingface import HuggingFaceEmbeddings #Ejecución local
#from langchain_community.embeddings import HuggingFaceHubEmbeddings #Legacy
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings #Ejecución en servidores HuggingFace
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain.document_loaders import TextLoader

c:\code\python\SETI\GenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and process documents

Load documents to do question answering over. If you want to do this over your documents, this is the section you should replace.

Next we split documents into small chunks. This is so we can find the most relevant chunks for a query and pass only those into the LLM.

In [ ]:
# Load and process the text
loader = TextLoader('state_of_the_union.txt',encoding='UTF-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, )
texts = text_splitter.split_documents(documents)
len(texts)

In [ ]:
print(texts[4])
len(texts[4].page_content) 

## Initialize PeristedChromaDB

Create embeddings for each chunk and insert into the Chroma vector database. The `persist_directory` argument tells ChromaDB where to store the database when it's persisted. 

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'ChromaDB'

embedding = HuggingFaceEndpointEmbeddings()
vectordb = Chroma.from_documents(documents=texts[0:30], embedding=embedding, persist_directory=persist_directory)

In [ ]:
vectordb.similarity_search("How can we react?")


In [ ]:
vectordb.max_marginal_relevance_search("How can we react?")

In [ ]:
vectordb.add_documents(texts[31:60])

In [ ]:
vectordb.g